In [32]:
import dask
import os
import re
import datetime
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import importlib

from letkf_forecasting import analyse_results, letkf_io

In [54]:
importlib.reload(analyse_results)
importlib.reload(letkf_io)

base_folder = '/a2/uaren/travis'

decimals = 2

year = 2014

month = 4
day = 15
day_name = 'translation'

# month = 5
# day = 29
# day_name = 'more_complex'

# month = 4
# day = 26
# day_name = 'two_levels'

dates = [datetime.date(2014, month, day)]
runs = ['owp_opt', 'owp_opt_anly_fore', 'persistence',
        'opt_flow', 'wrf_no_div', 'wrf_mean', 'radiosonde']
# runs = [ 'owp_opt', 'persistence', 'opt_flow', 'wrf_no_div',
#         'opt_flow_with_div', 'wrf', 'radiosonde', 'wrf_mean']
# runs = ['wrf_mean']

In [55]:
save_directory = "/home2/travis/python_code/letkf_forecasting/tables/"

In [56]:
horizons = [15, 30, 45, 60]

In [57]:
legend_dict = {'opt_flow': 'Opt.~Flow', 
               'opt_flow_with_div': 'Opt. Flow w/ Div.',
               'wrf_no_div': 'NWP Winds',
               'wrf': 'NWP w/ Div.',
               'owp_opt': 'ANOC',                                                                                
               'persistence': 'Persis.',
               'radiosonde': 'Radiosonde',
               'wrf_mean': 'NWP Avg.~Winds',
               'ens_member': 'Ens.~Member',
               'owp_opt_anly_fore': 'Anly. Fore.'} 

In [58]:
rmse = pd.DataFrame(index=horizons, columns=runs)
rmse.index.name = 'Horizon'
correlation = rmse.copy()
bias = rmse.copy()
for run_name in runs:
    anly_fore_flag = False
    modifier = ''
    dir_run_name = run_name
    if run_name == 'owp_opt_anly_fore':
        modifier = '_anly_fore'
        dir_run_name = 'owp_opt'
        anly_fore_flag = True
    results_folder_path = os.path.join(                                          
        base_folder,                                                               
        'results',                                                               
        f'{year:04}',                                                            
        f'{month:02}',                                                           
        f'{day:02}',                                                             
        dir_run_name)
    results_folder_path = letkf_io.find_latest_run(results_folder_path)
    results_folder_path = os.path.join(results_folder_path, 'single_day')
    
    stat_name = 'rmse' + modifier
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    rmse[run_name] = pd.read_hdf(file_path, stat_name)
    
    stat_name = 'bias' + modifier
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    bias[run_name] = pd.read_hdf(file_path, stat_name)
    
    stat_name = 'correlation' + modifier
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    correlation[run_name] = pd.read_hdf(file_path, stat_name)

In [59]:
peices = [rmse, correlation, bias]
combined = pd.concat(peices, axis=0,
                     keys=['RMSE', 'Corr.', 'Bias'])
combined = combined.rename(columns=legend_dict)

In [60]:
def is_empty(str):
    return str != ''

In [61]:
def format_table(text, header_num=5, footer_num=2):
    text = text.split(' ')
    text = list(filter(is_empty, text))
    text = ' '.join(text)
    split_text = text.split('\n')
    split_titles2 = split_text[2]
    removed = split_titles2[-2:]
    split_titles2 = split_titles2[:-2]
    split_titles2 = split_titles2.split('&')
    for count, this in enumerate(split_titles2):
        if len(this) > 2:
            this = this[0] + '{' + this[1:-1] + '}' + this[-1]
            split_titles2[count] = this
    split_text[2] = '&'.join(split_titles2) + removed
#     split_titles2 = split_text[2].split('&')
#     split_titles3 = split_text[3].split('&')
#     for num, title in enumerate(split_text[2].split('&')):
#         if ' w/ Div.' in title:
#             split_titles2[num] = title[:-8]
#             split_titles3[num] = title[-9:]
#     split_titles2 = '&'.join(split_titles2)
#     split_titles3 = '&'.join(split_titles3)
#     split_text[2] = split_titles2
#     split_text[3] = split_titles3
    
    for line_num, line in enumerate(split_text[header_num:-footer_num - 1]):
        split_line = line.split(' ')
        if split_line[0] == 'Corr.':
            Corr = True
        elif split_line[0] != '':
            Corr = False
        num_slice = slice(4, None, 2)
        numbers_str = split_line[num_slice]
        numbers = np.array(
            split_line[num_slice],
            dtype='float')
        if Corr:
            best_num = numbers.max()
        else:
            best_num = numbers[np.abs(numbers).argmin()]
        argmins = np.where(numbers == best_num)[0]
#         numbers = list(numbers.astype('str'))
        for argmin in argmins:
            numbers_str[argmin] = '\\B ' + numbers_str[argmin]
#             numbers_str[argmin] = '\\textbf{' + numbers_str[argmin] + '}'
        split_line[num_slice] = numbers_str
        split_text[header_num + line_num] = ' '.join(split_line)
    
#     for count in range(hor_num - 1):
#         split_line.insert(((count + 1)*run_num)*2 + 2 + count, '&')
#     split_line

    
    return '\n'.join(split_text)

In [62]:
column_format = 'll' + 'S[table-format=-1.3]' * len(runs)
text = combined.round(decimals=decimals).to_latex(column_format=column_format)
text2 = format_table(text)
text2 = re.sub('\\\\textasciitilde', '~', text2, count=5)
print(text2)

\begin{tabular}{llS[table-format=-1.3]S[table-format=-1.3]S[table-format=-1.3]S[table-format=-1.3]S[table-format=-1.3]S[table-format=-1.3]S[table-format=-1.3]}
\toprule
 & & {ANOC} & {Anly. Fore.} & {Persis.} & {Opt.~Flow} & {NWP Winds} & {NWP Avg.~Winds} & {Radiosonde} \\
{} & Horizon & & & & & & & \\
\midrule
RMSE & 15 & \B 0.03 & \B 0.03 & 0.07 & 0.04 & 0.04 & 0.04 & 0.07 \\
 & 30 & \B 0.04 & \B 0.04 & 0.07 & 0.05 & 0.05 & 0.05 & 0.08 \\
 & 45 & \B 0.05 & \B 0.05 & 0.08 & 0.06 & 0.06 & 0.06 & 0.09 \\
 & 60 & \B 0.05 & 0.06 & 0.09 & 0.06 & 0.07 & 0.06 & 0.09 \\
Corr. & 15 & \B 0.85 & 0.84 & 0.31 & 0.80 & 0.79 & 0.82 & 0.29 \\
 & 30 & \B 0.77 & 0.73 & 0.29 & 0.70 & 0.62 & 0.67 & 0.20 \\
 & 45 & \B 0.68 & 0.63 & 0.22 & 0.53 & 0.49 & 0.55 & 0.06 \\
 & 60 & \B 0.64 & 0.57 & 0.06 & 0.48 & 0.37 & 0.46 & 0.08 \\
Bias & 15 & \B -0.00 & \B -0.00 & -0.01 & \B -0.00 & \B -0.00 & \B -0.00 & \B -0.00 \\
 & 30 & \B -0.00 & \B -0.00 & -0.01 & \B -0.00 & -0.01 & \B -0.00 & \B -0.00 \\
 & 45 & \B -0.

In [63]:
this_file = os.path.join(save_directory, f'{day_name}_results.tex')
with open(this_file, 'w') as file:
    file.write(text2)